# Holmboe Instability

We will represent an incompressible, viscous, diffusive, Boussinesq, stratified fluid to simulate a Holmboe instability.
This exercise is very similar to the K-H instabilities. You need to find the right set up (boundary and initial conditions) to trigger a Holmboe instability. See class notes to find resources that will help you choose those parameters.

<img src="./Holmboe.png" width="600" height="300" />



This exercise was designed for the course Waves and Instabilities in Geophysical Fluid Dynamics of the Master's Degree in Advanced Physics and Applied Mathematics, at University of the Balearic Islands (Spain).

Author: Daniel Argüeso
Email: d.argueso@uib.es

Feb-2023

## Import modules

In [ ]:
## Import modules
import numpy as np
from dedalus import public as de
from dedalus.extras import flow_tools
import matplotlib.pyplot as plt
import h5py
import time

## Import and set logging

In [ ]:
import logging
root = logging.root
for h in root.handlers:
    h.setLevel("INFO")
logger = logging.getLogger(__name__)

## Define the problem

### Set problem domain

You need to define the domain of the problem. The first two items indicate the aspect ratio. For exmaple (2,1). The second one indicates the number of grid points in each direction. They should be consistent with the aspect ratio

In [ ]:
Lx, Ly = 
nx, ny = 

### Create bases and domain

They basically define the transformation between the grid space and the spectral space. There are various types of basis, but the most popular are:
- Fourier: to define periodic functions in an intervarl (usually the direction of the flow)
- Chebyshev: general functions in an interval (they require boundary conditions, usually top and bottom)

For each basis we specify the direction, the dimensions, the interval and the dealising. Dealising is used to evaluate operators in the Fourier space and for numerical stability. We use the default 3/2.

Then, the domain, which combines both bases and the dimensions above to define the problem domain

In [ ]:
x_basis = de.Fourier('x', nx, interval = (0,Lx), dealias =3/2)
y_basis = de.Chebyshev('y', ny, interval=(-Ly/2, Ly/2), dealias=3/2)
domain = de.Domain([x_basis,y_basis], grid_dtype=np.float64)

### Set parameters

In [ ]:
Reynolds = 
g = 

### Define the problem and the equations

We define the problem. We have different options, but we will use Initial value problem (IVP) in all of our exercises.
When we define the problem, we have to specify the domain, the variables, the parameters and the equations.

In the first command, we take the domain specifications we defined before and we define the variables that the problem will use. Here, for example we have pressure, horizontal and vertical velocities, and density. On top of that we also have vertical derivatives of density, and velocities, which we will need to specify the equations.

In [ ]:
problem = de.IVP(domain, variables = [])


In [ ]:
problem.parameters['Re'] = Reynolds
problem.parameters['g'] = g

These are the problem equations:

**Equations**

Next we will define the equations that will be solved on this domain.  The equations are

$$ \partial_t u + \boldsymbol{u}\boldsymbol{\cdot}\boldsymbol{\nabla} u + \frac{\partial_x p}{\rho_0} =  \frac{1}{{\rm Re}} \nabla^2 u $$
$$ \partial_t v + \boldsymbol{u}\boldsymbol{\cdot}\boldsymbol{\nabla} v + \frac{\partial_y p}{\rho_0} + \frac{\rho g}{\rho_0} =  \frac{1}{{\rm Re}} \nabla^2 v $$
$$ \boldsymbol{\nabla}\boldsymbol{\cdot}\boldsymbol{u} = 0 $$
$$ \partial_t \rho + \boldsymbol{u}\boldsymbol{\cdot}\boldsymbol{\nabla} \rho = 0 $$

The equations are written such that the left-hand side (LHS) is treated implicitly, and the right-hand side (RHS) is treated explicitly.  The LHS is limited to only linear terms, though linear terms can also be placed on the RHS.  Since $y$ is our special direction in this example, we also restrict the LHS to be at most first order in derivatives with respect to $y$.


In [ ]:
#[ADD EQUATIONS]

### Define the boundary conditions

As a general rule, for every derivate on y (our special dimension, Chebyshev), we need to add one boundary conditions. 
One of them is the pressure gauge.

Here you will need 5 different boundary conditions, including hte pressure gauge.

In [ ]:
#[ADD BDY CONDS]
# They can be of two types: Dirichlet or Neumann. Perhaps Neumann is better for the top and bottom walls?

## Define the solver

### Timestepping

We have different numerical schemes we can choose from. In our examples, we will use the RK443, but feel free to try others. You may read the documentation to see the full range of options.

In [ ]:
ts = de.timesteppers.RK443

### Building the solver

Here we simply initialize the solver.

In [ ]:
solver =  problem.build_solver(ts)

x = domain.grid(0)
y = domain.grid(1)
u = solver.state['u']
uy = solver.state['uy']
v = solver.state['v']
vy = solver.state['vy']
p = solver.state['p']
rho = solver.state['rho']

Set the solver parameters

Here we define some paramters, to help stop the simulation. In our case, we define the maximum duration, but we can define others.

In [ ]:
solver.stop_sim_time = 
solver.stop_wall_time = 
solver.stop_iteration = 

Set initial timestep and CFL conditions

We set the initial timestep, which will be later modified to ensure stability and optimize the simulation depending on the problem itself. We can also define cfl conditions based on velocities.

In [ ]:
# Add initial timestep and CFL CONDITIONS

### Initial conditions

Once the problem and the solver are set, we need to describe the initial conditions. These are critical to the problem and may be the difference between success and failure. A right choice of the initial conditions will produce the instability we are studying or simply make the model crash.

Set initial conditions with a sinusoidal perturbation in vertical velocity
**This step is crucial to obtain a Holmboe instability** If not set adequately, you will likely get a K-H instability.

In [ ]:
#Set intiial conditions using np.tanh for u and rho and np.exp * np.sin for v

u['g'] = 
rho['g'] = 
v['g'] = 

## Solving

In this step run the solver. At the same time we save some information for analysis we may want to make later on. And draw the plots every certain number of timesteps. Saving the analysis is not necessary, but it may be helpful to modify plots without having to run the entire simulation again.

Prepare the variables that will be saved for analysis (this is optional)

In [ ]:
analysis = solver.evaluator.add_file_handler('analysis_tasks', sim_dt=0.1, max_writes=50)
# ADD analysis tasks ( velocity, density)


### Plotting initial state

Make the plot for the initial state
(remember to use dealias before plotting, see R-T example)

In [ ]:
#plot initial state

### Move into solving loop

In [ ]:
#Add logger, 
#Initialize timestep,
# Initialize solver with while loop
# Create plot every N iterations


### ending program with information

In [ ]:
end_time = time.time()
logger.info('Run time: %f' %(end_time-start_time))
logger.info('Iterations: %i' %solver.iteration)